# Lab 1 - Getting Started

## Project Overview

In this project, you will use both `polars` and `pyspark` to process the data from the MinneMUDAC 2016 competition Dive into Water Data.  While the MinneMUDAC 2016 site, is no longer live, a copy was obtained using the [Wayback Machine (https://web.archive.org) and has been provided in [the overview notebook](./MinneMUDAC_2016_Overview.ipynb).  You should document your work in a Jupyter notebook, which will be used to submit your solution.

## Lab 1 Tasks

In this lab, you will perform the following tasks

1. Download and unzip the data.
2. Investigating the columns in various property data files.

### Task 1 - Data download and unzip

While the download links on the original site no longer work, you can access the data by clicking on [this link](https://mnscu-my.sharepoint.com/:u:/g/personal/bn8210wy_minnstate_edu/EdUePet8JsdKv5aUt9gvjoMBxQhXrOx73WpQyVNwLVDfkA?e=rR8qrc)

1. Move the zip file unto your repository
2. Unzip and move the files into your data folder.

**Hint.** Take a look the the Colab section of any module 5 lecture for an example.

In [2]:
import polars as pl
import polars.selectors as cs
from glob import glob
import requests, io, zipfile, os

In [6]:
%pip install bash

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for bash: filename=bash-0.6-py3-none-any.whl size=3005 sha256=74ad763664f4d601aa30ce3177457656787eaae4e09d423a16ca3eb7d4216ac7
  Stored in directory: c:\users\gpres\appdata\local\pip\cache\wheels\6d\fb\d1\e182674b941db8f914bac2a13f040509795032acc2961a7c7e
Successfully built bash
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: Building 'bash' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'bash'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [ ]:
# Perform the tasks listed above then use `glob` to verify the contents/paths.
glob('./data/raw_data/*')


['./data/raw_data\\2002_metro_tax_parcels.txt',
 './data/raw_data\\2003_metro_tax_parcels.txt',
 './data/raw_data\\2004_metro_tax_parcels.txt',
 './data/raw_data\\2005_metro_tax_parcels.txt',
 './data/raw_data\\2006_metro_tax_parcels.txt',
 './data/raw_data\\2007_metro_tax_parcels.txt',
 './data/raw_data\\2008_metro_tax_parcels.txt',
 './data/raw_data\\2009_metro_tax_parcels.txt',
 './data/raw_data\\2010_metro_tax_parcels.txt',
 './data/raw_data\\2011_metro_tax_parcels.txt',
 './data/raw_data\\2012_metro_tax_parcels.txt',
 './data/raw_data\\2013_metro_tax_parcels.txt',
 './data/raw_data\\2014_metro_tax_parcels.txt',
 './data/raw_data\\2015_metro_tax_parcels.txt',
 './data/raw_data\\mces_lakes_1999_2014.txt',
 './data/raw_data\\Parcel_Lake_Monitoring_Site_Xref.txt']

#### Questions

1. Notice that we have multiple property files, one per year.  What verb(s) will be used to combine these files?
2. Why is it important to compare the columns of these files?
3. Use the `%%bash` cell magic along with `head path` to inspect the first few lines of one of the files, where `...` is replaced with the specific path to one of the files.  How are the columns separated?

> <font color="orange"> 1. concat or union. 

2. Files from different years may have different column names/spellings. May have different delimiters
3. For 2002 parcels, | is the seperator.</font>

In [16]:
%%bash

head ./data/raw_data/2004_metro_tax_parcels.txt

ACRES_DEED|ACRES_POLY|AGPRE_ENRD|AGPRE_EXPD|AG_PRESERV|BASEMENT|BLDG_NUM|BLOCK|CITY|CITY_USPS|COOLING|COUNTY_ID|DWELL_TYPE|EMV_BLDG|EMV_LAND|EMV_TOTAL|FIN_SQ_FT|GARAGE|GARAGESQFT|GREEN_ACRE|HEATING|HOMESTEAD|HOME_STYLE|ID|LANDMARK|LOT|MULTI_USES|NUM_UNITS|OPEN_SPACE|OWNER_MORE|OWNER_NAME|OWN_ADD_L1|OWN_ADD_L2|OWN_ADD_L3|PARC_CODE|PIN|PLAT_NAME|PREFIXTYPE|PREFIX_DIR|SALE_DATE|SALE_VALUE|SCHOOL_DST|SPEC_ASSES|STREETNAME|STREETTYPE|SUFFIX_DIR|Shape_Area|Shape_Leng|TAX_ADD_L1|TAX_ADD_L2|TAX_ADD_L3|TAX_CAPAC|TAX_EXEMPT|TAX_NAME|TOTAL_TAX|UNIT_INFO|USE1_DESC|USE2_DESC|USE3_DESC|USE4_DESC|WSHD_DIST|XUSE1_DESC|XUSE2_DESC|XUSE3_DESC|XUSE4_DESC|YEAR_BUILT|Year|ZIP|ZIP4|centroid_lat|centroid_long
0.0|8.03|||N||||SAINT FRANCIS|||003||0.0|17750.0|23964.0|0.0|||N||N|||||||N|||24457 DOGWOOD ST NW|BETHEL|MN,  55005|0.0|003-253424110001|||||0.0|15|0.0||||||24457 DOGWOOD ST NW|BETHEL|MN,  55005|351.0|N||589.0||||||UPPER RUM RIVER WMO|||||1980.0|2004|||45.41332|-93.26739
0.0|0.93|||N||24457||SAINT FRANCI

## Task 2 - Perform a column exploration on the property files

<img src="./img/column_master_file.png" width="800">

**Hints.**

1. Use `glob` to get a list of all the parcel files.
2. Use a list comprehension contains pairs of value of the form `(year, df)` where `df` a `polars` or `pyspark` data frame for each file. Keep in mind that the files are large so we will need to leverage the lazy natures of `polars` and `pyspark`. You will want to use `scan_csv` in `polars`.  On the other hand, `pyspark` is lazy and will do minimal work on this step.
3. Perform a column exploration by creating an indicator summary table, with on row per column and one column per year/file.  The values in the table should be `1` if the column is present in that year's file and `0` otherwise.  You may need to pivot or reshape your data frame to get it into this format.
4. There is one problematic year.  Which year is it and what is the problem?  We will be skipping this year in future analysis.
5. Sort the summary table by the column names and inspect the results.  In particular, look for similar names with different spellings or capitalizations.
6. Identify the common columns that are present in all years.

In [9]:
# Glob/path processing code here
parcel_files = glob('./data/raw_data/20*.txt')
parcel_files

['./data/raw_data\\2002_metro_tax_parcels.txt',
 './data/raw_data\\2003_metro_tax_parcels.txt',
 './data/raw_data\\2004_metro_tax_parcels.txt',
 './data/raw_data\\2005_metro_tax_parcels.txt',
 './data/raw_data\\2006_metro_tax_parcels.txt',
 './data/raw_data\\2007_metro_tax_parcels.txt',
 './data/raw_data\\2008_metro_tax_parcels.txt',
 './data/raw_data\\2009_metro_tax_parcels.txt',
 './data/raw_data\\2010_metro_tax_parcels.txt',
 './data/raw_data\\2011_metro_tax_parcels.txt',
 './data/raw_data\\2012_metro_tax_parcels.txt',
 './data/raw_data\\2013_metro_tax_parcels.txt',
 './data/raw_data\\2014_metro_tax_parcels.txt',
 './data/raw_data\\2015_metro_tax_parcels.txt']

In [13]:
year_dfs = [(int(f.split('\\')[-1][:4]), pl.scan_csv(f, separator='|')) for f in parcel_files]
year_dfs

#notes : For each file path in parcel_files, this code:

#Extracts the year from the filename (first 4 characters after the last \)
#Creates a lazy Polars DataFrame by scanning the CSV file with pipe | as the separator

[(2002, <LazyFrame at 0x2141E290EC0>),
 (2003, <LazyFrame at 0x2141E866D50>),
 (2004, <LazyFrame at 0x2141E867110>),
 (2005, <LazyFrame at 0x2141E01EEA0>),
 (2006, <LazyFrame at 0x2141E01ED70>),
 (2007, <LazyFrame at 0x2141E0CD010>),
 (2008, <LazyFrame at 0x2141E851590>),
 (2009, <LazyFrame at 0x2141E8516A0>),
 (2010, <LazyFrame at 0x2141EB6D550>),
 (2011, <LazyFrame at 0x2141EB6D650>),
 (2012, <LazyFrame at 0x2141E9597C0>),
 (2013, <LazyFrame at 0x2141E9599A0>),
 (2014, <LazyFrame at 0x2141E672A50>),
 (2015, <LazyFrame at 0x2141E672B30>)]

In [14]:
column_data = []
for year, df in year_dfs:
    for col in df.columns:
        column_data.append({'column': col, 'year': year, 'present': 1})

column_summary = (
    pl.DataFrame(column_data)
    .pivot(values='present', index='column', columns='year')
    .fill_null(0)
)
pl.Config.set_tbl_rows(-1)
pl.Config.set_tbl_cols(-1)
column_summary.sort('column')


C:\Users\nw5125ic\AppData\Local\Temp\ipykernel_14752\1620467011.py:3: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  for col in df.columns:
C:\Users\nw5125ic\AppData\Local\Temp\ipykernel_14752\1620467011.py:8: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  .pivot(values='present', index='column', columns='year')


column,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""ACRES_DEED""",1,0,1,1,1,1,1,1,1,1,1,1,1,1
"""ACRES_POLY""",1,0,1,1,1,1,1,1,1,1,1,1,1,1
"""AGPRE_ENRD""",1,0,1,1,1,1,1,1,1,1,1,1,1,1
"""AGPRE_EXPD""",1,0,1,1,1,1,1,1,1,1,1,1,1,1
"""AG_PRESERV""",1,0,1,1,1,1,1,1,1,1,1,1,1,1
"""BASEMENT""",1,0,1,1,1,1,1,1,1,1,1,1,1,1
"""BLDG_NUM""",1,1,1,1,1,1,1,1,1,1,1,1,1,1
"""BLOCK""",1,0,1,1,1,1,1,1,1,1,1,1,1,1
"""CITY""",1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [15]:
# Column names that are same across all years
common_columns = column_summary.filter(pl.all_horizontal(pl.all().exclude('column') == 1)).select('column')
common_columns

column
str
"""BLDG_NUM"""
"""CITY"""
"""COUNTY_ID"""
"""EMV_BLDG"""
"""EMV_LAND"""
"""EMV_TOTAL"""
"""HOMESTEAD"""
"""NUM_UNITS"""
"""OWN_ADD_L1"""


In [16]:
# 2004+ , common columns
column_data_filtered = []
for year, df in year_dfs:
    if year >= 2004:
        for col in df.columns:
            column_data_filtered.append({'column': col, 'year': year, 'present': 1})

common_columns_2004_plus = (
    pl.DataFrame(column_data_filtered)
    .pivot(values='present', index='column', columns='year')
    .fill_null(0)
    .filter(pl.all_horizontal(pl.all().exclude('column') == 1))
    .sort('column')
)

common_columns_2004_plus

C:\Users\nw5125ic\AppData\Local\Temp\ipykernel_14752\3681834538.py:5: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  for col in df.columns:
C:\Users\nw5125ic\AppData\Local\Temp\ipykernel_14752\3681834538.py:10: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  .pivot(values='present', index='column', columns='year')


column,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""ACRES_DEED""",1,1,1,1,1,1,1,1,1,1,1,1
"""ACRES_POLY""",1,1,1,1,1,1,1,1,1,1,1,1
"""AGPRE_ENRD""",1,1,1,1,1,1,1,1,1,1,1,1
"""AGPRE_EXPD""",1,1,1,1,1,1,1,1,1,1,1,1
"""AG_PRESERV""",1,1,1,1,1,1,1,1,1,1,1,1
"""BASEMENT""",1,1,1,1,1,1,1,1,1,1,1,1
"""BLDG_NUM""",1,1,1,1,1,1,1,1,1,1,1,1
"""BLOCK""",1,1,1,1,1,1,1,1,1,1,1,1
"""CITY""",1,1,1,1,1,1,1,1,1,1,1,1


In [17]:
columns_to_keep = common_columns_2004_plus.select('column').to_series().to_list()
columns_to_keep

['ACRES_DEED',
 'ACRES_POLY',
 'AGPRE_ENRD',
 'AGPRE_EXPD',
 'AG_PRESERV',
 'BASEMENT',
 'BLDG_NUM',
 'BLOCK',
 'CITY',
 'CITY_USPS',
 'COOLING',
 'COUNTY_ID',
 'DWELL_TYPE',
 'EMV_BLDG',
 'EMV_LAND',
 'EMV_TOTAL',
 'FIN_SQ_FT',
 'GARAGE',
 'GARAGESQFT',
 'GREEN_ACRE',
 'HEATING',
 'HOMESTEAD',
 'HOME_STYLE',
 'LANDMARK',
 'LOT',
 'MULTI_USES',
 'NUM_UNITS',
 'OPEN_SPACE',
 'OWNER_MORE',
 'OWNER_NAME',
 'OWN_ADD_L1',
 'OWN_ADD_L2',
 'OWN_ADD_L3',
 'PARC_CODE',
 'PIN',
 'PLAT_NAME',
 'PREFIXTYPE',
 'PREFIX_DIR',
 'SALE_DATE',
 'SALE_VALUE',
 'SCHOOL_DST',
 'SPEC_ASSES',
 'STREETNAME',
 'STREETTYPE',
 'SUFFIX_DIR',
 'Shape_Area',
 'Shape_Leng',
 'TAX_ADD_L1',
 'TAX_ADD_L2',
 'TAX_ADD_L3',
 'TAX_CAPAC',
 'TAX_EXEMPT',
 'TAX_NAME',
 'TOTAL_TAX',
 'UNIT_INFO',
 'USE1_DESC',
 'USE2_DESC',
 'USE3_DESC',
 'USE4_DESC',
 'WSHD_DIST',
 'XUSE1_DESC',
 'XUSE2_DESC',
 'XUSE3_DESC',
 'XUSE4_DESC',
 'YEAR_BUILT',
 'Year',
 'ZIP',
 'ZIP4',
 'centroid_lat',
 'centroid_long']

In [1]:
from column_data import columns_to_keep

columns_to_keep

['ACRES_DEED',
 'ACRES_POLY',
 'AGPRE_ENRD',
 'AGPRE_EXPD',
 'AG_PRESERV',
 'BASEMENT',
 'BLDG_NUM',
 'BLOCK',
 'CITY',
 'CITY_USPS',
 'COOLING',
 'COUNTY_ID',
 'DWELL_TYPE',
 'EMV_BLDG',
 'EMV_LAND',
 'EMV_TOTAL',
 'FIN_SQ_FT',
 'GARAGE',
 'GARAGESQFT',
 'GREEN_ACRE',
 'HEATING',
 'HOMESTEAD',
 'HOME_STYLE',
 'LANDMARK',
 'LOT',
 'MULTI_USES',
 'NUM_UNITS',
 'OPEN_SPACE',
 'OWNER_MORE',
 'OWNER_NAME',
 'OWN_ADD_L1',
 'OWN_ADD_L2',
 'OWN_ADD_L3',
 'PARC_CODE',
 'PIN',
 'PLAT_NAME',
 'PREFIXTYPE',
 'PREFIX_DIR',
 'SALE_DATE',
 'SALE_VALUE',
 'SCHOOL_DST',
 'SPEC_ASSES',
 'STREETNAME',
 'STREETTYPE',
 'SUFFIX_DIR',
 'Shape_Area',
 'Shape_Leng',
 'TAX_ADD_L1',
 'TAX_ADD_L2',
 'TAX_ADD_L3',
 'TAX_CAPAC',
 'TAX_EXEMPT',
 'TAX_NAME',
 'TOTAL_TAX',
 'UNIT_INFO',
 'USE1_DESC',
 'USE2_DESC',
 'USE3_DESC',
 'USE4_DESC',
 'WSHD_DIST',
 'XUSE1_DESC',
 'XUSE2_DESC',
 'XUSE3_DESC',
 'XUSE4_DESC',
 'YEAR_BUILT',
 'Year',
 'ZIP',
 'ZIP4',
 'centroid_lat',
 'centroid_long']

In [19]:
column_schema = {col: pl.String for col in columns_to_keep}
column_schema

{'ACRES_DEED': String,
 'ACRES_POLY': String,
 'ADDITION': String,
 'AGPRE_ENRD': String,
 'AGPRE_EXPD': String,
 'BLOCK': String,
 'BLDG_NUM': String,
 'CITY': String,
 'COUNTY_ID': String,
 'DEEDBOOK': String,
 'DEEDPAGE': String,
 'EMV_LAND': String,
 'EMV_TOTAL': String,
 'FIN_SQ_FT': String,
 'GARAGE': String,
 'GARAGESQFT': String,
 'GREEN_ACRE': String,
 'HOMESTEAD': String,
 'LANDUSE': String,
 'LANDUSE_B1': String,
 'LANDUSE_B2': String,
 'LANDUSE_B3': String,
 'LANDUSE_B4': String,
 'LOT': String,
 'MUNI_CODE': String,
 'MUNI_NAME': String,
 'NET_TAX_CAP': String,
 'NUM_UNITS': String,
 'OPEN_SPACE': String,
 'PARC_CODE': String,
 'PARCEL_ID': String,
 'PIN': String,
 'PLAT_NAME': String,
 'PREFIXTYPE': String,
 'PREFIX_DIR': String,
 'SALE_DATE': String,
 'SALE_VALUE': String,
 'SCHOOL_DST': String,
 'SPEC_ASSES': String,
 'STREETNAME': String,
 'STREETTYPE': String,
 'STRUCTURE': String,
 'SUFFIX_DIR': String,
 'TAX_CAPNET': String,
 'TAX_NAME': String,
 'TOTAL_TAX': String

In [21]:
from column_data import columns_to_keep, column_schema

column_schema

{'ACRES_DEED': String,
 'ACRES_POLY': String,
 'ADDITION': String,
 'AGPRE_ENRD': String,
 'AGPRE_EXPD': String,
 'BLOCK': String,
 'BLDG_NUM': String,
 'CITY': String,
 'COUNTY_ID': String,
 'DEEDBOOK': String,
 'DEEDPAGE': String,
 'EMV_LAND': String,
 'EMV_TOTAL': String,
 'FIN_SQ_FT': String,
 'GARAGE': String,
 'GARAGESQFT': String,
 'GREEN_ACRE': String,
 'HOMESTEAD': String,
 'LANDUSE': String,
 'LANDUSE_B1': String,
 'LANDUSE_B2': String,
 'LANDUSE_B3': String,
 'LANDUSE_B4': String,
 'LOT': String,
 'MUNI_CODE': String,
 'MUNI_NAME': String,
 'NET_TAX_CAP': String,
 'NUM_UNITS': String,
 'OPEN_SPACE': String,
 'PARC_CODE': String,
 'PARCEL_ID': String,
 'PIN': String,
 'PLAT_NAME': String,
 'PREFIXTYPE': String,
 'PREFIX_DIR': String,
 'SALE_DATE': String,
 'SALE_VALUE': String,
 'SCHOOL_DST': String,
 'SPEC_ASSES': String,
 'STREETNAME': String,
 'STREETTYPE': String,
 'STRUCTURE': String,
 'SUFFIX_DIR': String,
 'TAX_CAPNET': String,
 'TAX_NAME': String,
 'TOTAL_TAX': String

#### Your conclusions

<font color="orange">
2003 is a problem year, lots of columns there dont match the other years.

We have 70 common columns in 2004+</font>